In [ ]:
%pip install -q bqplot==0.12.40 ipyleaflet==0.17.3 ipywidgets==8.0.6 jupyterlab_widgets==3.0.7

# Recent fires

Showing large fires that were burning when the fire atlas was last updated. Only includes fires with an area greater than 2. Testing making a change.

You can learn more about how to access [this data](https://firenrt.delta-backend.com/collections/public.eis_fire_snapshot_perimeter_nrt) directly in [this VEDA documentation page](https://nasa-impact.github.io/veda-docs/notebooks/tutorials/mapping-fires.html).

In [ ]:
import json
import urllib

import datetime as dt
import ipywidgets as widgets
import geopandas as gpd
import pandas as pd

from js import fetch

from bqplot import Lines, Figure, LinearScale, DateScale, Axis

from ipyleaflet import Map, WidgetControl, GeoData, LayersControl

In [ ]:
URL = "https://firenrt.delta-backend.com/collections/public.eis_fire_snapshot_perimeter_nrt"
res = await fetch(URL)
text = await res.text()

most_recent_time = json.loads(text)["extent"]["temporal"]["interval"][0][1]

In [ ]:
async def get_data(url):
    response = await fetch(url)
    text = await response.text()
    result = json.loads(text)
    assert result["numberMatched"] == result["numberReturned"], "Increase the limit"
    return gpd.GeoDataFrame.from_features(result["features"])

In [ ]:
farea_min = 2
duration_min = 0

# get fires that were burning in the most recent 24 hours
start = (dt.datetime.fromisoformat(most_recent_time) - dt.timedelta(days=1)).isoformat()
datetime_interval = f"{start}/.."

url = (
    'https://firenrt.delta-backend.com/'
    'collections/public.eis_fire_snapshot_perimeter_nrt/items?'
    f'datetime={urllib.parse.quote(datetime_interval)}&'
    'limit=1000&'
    f'filter=farea%3E{farea_min}+AND+duration%3E{duration_min}&'
    'f=geojson'
)

df = await get_data(url)

In [ ]:
fireids = ",".join(str(fireid) for fireid in df.fireid)
url = (
    "https://firenrt.delta-backend.com/collections/public.eis_fire_snapshot_fireline_nrt/items?"
    f"filter=fireid+IN+({fireids})&"
    "limit=1000"
)
flines = await get_data(url)

In [ ]:
def clean_data(data):
    data.fireid = data.fireid.astype(str)
    data.fireid = data.fireid.apply(lambda x: f'F{x}' if not x.startswith("F") else x)
    # data.t = pd.to_datetime(data.t)
    data = data.sort_values("t", ascending=False)
    return data

df = clean_data(df)
flines = clean_data(flines)
pins = df.copy()
pins["geometry"] = df.geometry.representative_point()

In [ ]:
fireids = ",".join(f"'{fireid}'" for fireid in df.fireid)
url = (
    "https://firenrt.delta-backend.com/collections/public.eis_fire_lf_perimeter_nrt/items?"
    f"filter=fireid+IN+({fireids})&"
    "limit=1000"
)
fires_df = await get_data(url)
fires_df = clean_data(fires_df)

# join this data with the current snapshot data
fires_df = pd.concat([df.drop(columns=["isactive"]), fires_df])
fires_df = fires_df.drop_duplicates()

In [ ]:
fireid = fires_df.fireid.value_counts().index[0]
data_name = 'farea'

data = fires_df
subset = data[data.fireid == fireid].sort_values("t")

y_data = subset[data_name].values
x_data = pd.to_datetime(subset["t"].values)
t = x_data.max()

In [ ]:
y_scale = LinearScale()
date_scale = DateScale()

lines = Lines(x=x_data, y=y_data, scales={'x': date_scale, 'y': y_scale})
v_lines = Lines(x=[t, t], y=[y_data.min(), y_data.max()], scales={'x': date_scale, 'y': y_scale}, colors=["red"])

ax_x = Axis(label='Time', scale=date_scale)
ax_y = Axis(label=data_name.capitalize(), scale=y_scale, orientation='vertical', side='left')

figure = Figure(axes=[ax_x, ax_y], title=fireid, marks=[lines, v_lines], animation_duration=500,
                layout={'max_height': '400px', 'width': '500px'})

In [ ]:
def update_figure(fireid, data_name, t):
    try:
        subset = data[data.fireid == fireid].sort_values("t")
        y_data = subset[data_name].values
        x_data = pd.to_datetime(subset["t"].values)
        lines.y = y_data
        lines.x = x_data
        v_lines.x = [t, t]
        v_lines.y = [lines.y.min(), lines.y.max()]
        ax_y.label = data_name.capitalize()
        figure.title = fireid
    except IndexError:
        pass
    
def update_y(fireid, data_name):
    try:
        subset = data[data.fireid == fireid].sort_values("t")
        lines.y = subset[data_name].values
        v_lines.y = [lines.y.min(), lines.y.max()]
        ax_y.label = data_name.capitalize()

    except IndexError:
        pass

In [ ]:
m = Map(center=(39, -98), zoom=4, scroll_wheel_zoom=True)

farea_min = 5
duration_min = 2

m.layout.min_height="800px"

current_points = GeoData(geo_dataframe=pins, name = 'Pins')

current_polygons = GeoData(
    geo_dataframe=df, 
    style={'fillColor': 'red','color': 'red'}, 
    hover_style={'fillColor': 'black'}, 
    name='Current Perimeters'
)

archive_polygons = GeoData(
    geo_dataframe=fires_df, 
    style={'color': 'black', "fillOpacity": 0}, 
    hover_style={'fillColor': 'black', "fillOpacity": 0.5}, 
    name='Archive Perimeters'
)

fline_paths = GeoData(
    geo_dataframe=flines,
    style={'color': 'orange'},
    name='Fire Lines'
)

layout = widgets.Layout(width = "300px")
style = {'description_width': 'initial'}

duration_min_control = widgets.FloatText(value=duration_min, min=0, step=.5, description='Minimum burn duration (days):', layout=layout, style=style)
farea_min_control = widgets.IntSlider(value=farea_min, min=0, step=5, max=100, description='Minimum fire area: ', layout=layout, style=style)

out = widgets.Output(layout=layout)

dropdown = widgets.Dropdown(
    options=['farea', 'n_pixels', 'flinelen'],
    value=data_name,
    description='Plotting:'
)

m.add(current_points)
m.add(current_polygons)
m.add(archive_polygons)
m.add(fline_paths)
m.add(LayersControl())
m.add(WidgetControl(widget=duration_min_control, position="topleft"))
m.add(WidgetControl(widget=farea_min_control, position="topleft"))
m.add(WidgetControl(widget=out, position="bottomleft"))
m.add(WidgetControl(widget=figure, position='bottomright'))
m.add(WidgetControl(widget=dropdown, position='bottomright'))

def display_properties(feature, **kwargs):
    out.clear_output()
    with out:
        display(pd.Series(feature["properties"]))
        
def on_hover(event, feature, **kwargs):
    global fireid
    global t
    
    fireid = feature['properties']['fireid']
    t = pd.to_datetime(feature['properties']['t'])

    update_figure(fireid, data_name, t)
    display_properties(feature)
    
    
def on_dropdown(change):
    global data_name

    data_name = change['new']
    update_y(fireid, data_name)

def filter_data():
    global farea_min
    global duration_min
    
    subset_idx = (df["farea"] >= farea_min) & (df["duration"] >= duration_min)
    subset = df[subset_idx]
    fireids = subset.fireid.unique().tolist()
    flines_subset = flines[flines.fireid.isin(fireids)]
    archive_subset = fires_df[fires_df.fireid.isin(fireids)]
    
    current_points.data = json.loads(pins[subset_idx].to_json())
    current_polygons.data = json.loads(subset.to_json())
    archive_polygons.data = json.loads(archive_subset.to_json())
    fline_paths.data = json.loads(flines_subset.to_json())
    

def on_change_farea(change):
    global farea_min
    
    farea_min = change["new"]
    filter_data()

def on_change_duration(change):
    global duration_min
    
    duration_min = change["new"]
    filter_data()

filter_data()

dropdown.observe(on_dropdown, 'value')
current_polygons.on_hover(on_hover)
archive_polygons.on_hover(on_hover)
current_points.on_click(on_hover)
farea_min_control.observe(on_change_farea, "value")
duration_min_control.observe(on_change_duration, "value")

m

⚠️ This visualization was created using a [jupyterlite-pyodide-kernel](https://github.com/jupyterlite/pyodide-kernel) and [voici](https://voici.readthedocs.io/en/latest/). It is rendered client-side (in your browser!) via the magic of [Wasm](https://webassembly.org/). This is an exciting and rapidly evolving space which means that this visualization is highly experimental and likely to break. ⚠️